# Time-Series Forecasting

In [ ]:
!pip install scikit-learn==1.0.2
!pip install xgboost
!pip install scikit-learn-intelex

### Import the dataset

In [ ]:
import pandas as pd

# Import dataset
data = pd.read_csv('energydata_complete.csv')

### Dataset Information

In [ ]:
# Provide information about the dataset
# Prints the head (5 first rows) and tail (5 last rows)
# and also prints the size of the dataset.
def dump_source_info(dataset):
  print(dataset.head())
  print("-----------------------")
  print(dataset.tail())
  print("-----------------------")
  print(dataset.shape)

### Variables

In [ ]:
# Prints all the different columns
# in the dataset and information
# about these columns.
def dump_variables(dataset):
  print(dataset.columns)
  print("-----------------------")
  print(dataset.info())
  print("-----------------------")
  print(dataset.describe())

### Time granularity

In [ ]:
print(data['date'].head())

By printing the date column, we observe that the sampling rate is done every 10 minutes for this dataset

### Duration

In [ ]:
# We calculate the duration by substracting from the maximum time
# index the minimum time index.
def dump_duration(dataset):
  data['date'] = pd.to_datetime(data['date'])

  # Get the minimum timestamp
  min_timestamp = data['date'].min()

  # Get the maximum timestamp
  max_timestamp = data['date'].max()

  # Calculate the duration
  duration = max_timestamp - min_timestamp

  # Print the duration
  print(duration)

In [ ]:
dump_duration(data)

**The duration is 137 days approximately 4.5 months.**

### Plotting the Dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the target variable
# We set it to be the column
# Appliances.
def plot_appliances(dataset):
  # Reset index and keep 'date' as a column
  series = dataset.reset_index()

  # Ensure 'date' is in datetime format
  series['date'] = pd.to_datetime(series['date'])

  plt.rcParams['figure.figsize'] = [12, 6]
  sns.set_theme(style='darkgrid')

  # Use 'date' column as x-axis
  sns.lineplot(data=series, x='date', y='Appliances')

  plt.ylabel('Appliances')
  plt.xlabel('date')
  plt.title('Applicancies time series')

  plt.xticks(rotation=45, ha='right')

  plt.show()

In [ ]:
plot_appliances(data)

### Dealing with missing values

In [ ]:
# Function that prints if there are any Nan values
# Also prints the count and where they are, if there
# are nan values.
def check_missing_values(series):
  if series.isnull().values.any():
    print("Are there any Nan values? ", series.isnull().values.any())
    print("The sum of the Nan values at each feature is\n:", series.isnull().sum())
    nan_check_df = series.isnull()
    print(f"Nan values: \n {nan_check_df}")
  else:
    print("Are there any Nan values? ", series.isnull().values.any())

In [ ]:
check_missing_values(data)

**We observe that the original data has no nan values.**

### Outliers

For outliers the IQR method is used

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,mean_absolute_percentage_error
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Function to replace outliers using IQR method
# and print the number of replaced outliers.
def replace_outliers_with_threshold(df, threshold=1):
    df_cleaned = df.copy()
    outliers_replaced = {}

    for col in df_cleaned.select_dtypes(include=[np.number]).columns:
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - (threshold * IQR)
        upper_bound = Q3 + (threshold * IQR)

        # Count the number of outliers before replacement
        outliers_before = ((df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)).sum()

        # Replace outliers with the threshold values (lower and upper bounds)
        df_cleaned[col] = np.where(df_cleaned[col] < lower_bound, lower_bound,
                                   np.where(df_cleaned[col] > upper_bound, upper_bound, df_cleaned[col]))

        # Count the number of outliers after replacement (should be zero)
        outliers_after = ((df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)).sum()

        # Store the number of outliers replaced for this column
        outliers_replaced[col] = outliers_before

        if outliers_before > 0:
            print(f"Column '{col}': {outliers_before} outliers replaced.")

    return df_cleaned, outliers_replaced


### ACF/PACF Plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ACF/PACF plot to check if after the decomposition
# some columns still have seasonality/trending issues.
def plot_acf_pacf(dataset, parameter, lags):

  for column in dataset.columns:
    plt.figure(figsize=(10, 6))

    if parameter == 'acf' and column == 'Appliances':
      plot_acf(dataset[column], lags=lags)
      plt.title(f"{column} {parameter}")
      plt.xlabel("Lag")
      plt.ylabel("Correlation")
      plt.legend()
      plt.xticks(rotation=45, ha='right')
      plt.show()
    elif parameter == 'pacf':
      plot_pacf(dataset[column], lags=lags)
      plt.title(f"{column} {parameter}")
      plt.xlabel("Lag")
      plt.ylabel("Correlation")
      plt.legend()
      plt.xticks(rotation=45, ha='right')
      plt.show()

In [ ]:
plot_acf_pacf(data, 'acf', 200)

from this plot we decide to add lags every :
- day (144)
- 2 days (288)
- 3 days (432)
- 12 hours (72)
- 6 hours (36)
- 3 hours (18)
- 2 hours (12)
- 1 hour (6)

we also decide to add rolling averages every:
- Approximately every 2 hours (10)
- Every 5 hours (30)
- Approximately every 8 hours (50)
- Every day (144)
- Every 2 days (288)

### Feature engineering

In [ ]:
# Function that adds lags as a column in the dataset
def add_periodic_lags(df, target_column, period=144):
    df_lagged = df.copy()
    df_lagged[f'{target_column}_lag_{period}'] = df_lagged[target_column].shift(period)

    return df_lagged

# Function that adds rolling averages in the dataset
def add_rolling_means(df, target_column, window_size=144):
    df_rolling = df.copy()
    df_rolling[f'{target_column}_rolling_mean_{window_size}'] = df_rolling[target_column].rolling(window=window_size).mean()

    return df_rolling

In [ ]:
# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Create NSM feature (Seconds from Midnight)
data['NSM'] = data['date'].dt.hour * 3600 + data['date'].dt.minute * 60 + data['date'].dt.second

# Create week_status feature (1 for weekday, 0 for weekend)
data['week_status'] = data['date'].dt.weekday.apply(lambda x: 1 if x < 5 else 0)

# One-hot encode day of the week(Monday to Sunday)
day_of_week_encoded = pd.get_dummies(data['date'].dt.day_name(), prefix='day')

# Combine the one-hot encoded columns with the original dataset
data = pd.concat([data, day_of_week_encoded], axis=1)


# Features as per the paper plus new features
features = [
    'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
    'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
    'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
    'Tdewpoint', 'NSM', 'week_status', 'rv1', 'rv2',
] + list(day_of_week_encoded.columns)

# Add periodic lags and rolling means on the dataset
data_new = add_periodic_lags(data, 'Appliances', 144)
data_new = add_periodic_lags(data_new, 'Appliances', 288)
data_new = add_periodic_lags(data_new, 'Appliances', 432)
data_new = add_periodic_lags(data_new, 'Appliances', 72)
data_new = add_periodic_lags(data_new, 'Appliances', 36)
data_new = add_periodic_lags(data_new, 'Appliances', 18)
data_new = add_periodic_lags(data_new, 'Appliances', 12)
data_new = add_periodic_lags(data_new, 'Appliances', 6)
data_new = add_rolling_means(data_new, 'Appliances', 10)
data_new = add_rolling_means(data_new, 'Appliances', 30)
data_new = add_rolling_means(data_new, 'Appliances', 50)
data_new = add_rolling_means(data_new, 'Appliances', 144)
data_new = add_rolling_means(data_new, 'Appliances', 288)

# Create a list with all the columns removing the random variables rv1 and rv2.
new_features = [
    'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
    'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
    'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
    'Tdewpoint', 'NSM', 'week_status',
    'Appliances_lag_144',
       'Appliances_lag_288', 'Appliances_lag_432', 'Appliances_lag_72',
       'Appliances_lag_36', 'Appliances_lag_18', 'Appliances_rolling_mean_10',
       'Appliances_rolling_mean_30', 'Appliances_rolling_mean_50',
       'Appliances_rolling_mean_144', 'Appliances_rolling_mean_288','Appliances_lag_6'
       ,'Appliances_lag_12'
] + list(day_of_week_encoded.columns)



target = 'Appliances'

# Drop irrelevant columns
# X = data[features]
X = data_new[new_features]
y = data[target]

# Because of the addition of lag features and rolling averages
# we decide to fill the nan values with the mean
X.fillna(X.mean(), inplace=True)

From reading the paper we decide to remove the lights Visibility and rv1, rv2 columns. The last two (rv1, rv2) are just random variables and therefore do not contribute to the accuracy of the models. Where we as well shaw that after removing the lights and Visibility got better results

In [ ]:
data_new = data_new.drop(columns=['lights', 'Visibility', 'rv1', 'rv2'])

In [ ]:
# Save the DataFrame as a CSV file
data_new.to_csv('energydata_complete_new.csv', index=False)

In [ ]:
check_missing_values(data_new)

In [ ]:
# Handle missing values after finding out they exist
X.fillna(X.mean(), inplace=True)

# Handle outliers using the function and print the count of outliers replaced
data_cleaned, outliers_replaced = replace_outliers_with_threshold(data_new)

# Prepare the new data for the columns we removed
new_features = data_new.columns.tolist()
new_features.remove('Appliances')

# Initialize the scaler 
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()  

# Get the new X and y for the outlier-removed dataset
X_cleaned = data_cleaned[new_features]
X_cleaned = X_cleaned.drop(columns=['date'])
y_cleaned = data_cleaned[target]

### Method to plot real vs predicted values

In [ ]:
def plot_forecast_vs_actual(y_true, y_pred, title="Forecasted vs Actual Values", num_samples=200):
    plt.figure(figsize=(12, 6))
    plt.plot(y_true.values[:num_samples], label="Actual", color='purple')
    plt.plot(y_pred[:num_samples], label="Forecasted", color='green')
    plt.title(title)
    plt.xlabel("Sample Index")
    plt.ylabel("Appliance Energy Consumption")
    plt.legend()
    plt.show()

In [ ]:
def plot_forecast_vs_actual_dl(y_true, y_pred, title="Forecasted vs Actual Values", num_samples=200):
    plt.figure(figsize=(12, 6))
    plt.plot(y_true[:num_samples], label="Actual", color='purple')
    plt.plot(y_pred[:num_samples], label="Forecasted", color='green')
    plt.title(title)
    plt.xlabel("Sample Index")
    plt.ylabel("Appliance Energy Consumption")
    plt.legend()
    plt.show()

### Method to plot the residuals

In [ ]:
def plot_residuals(y_true, y_pred, title="Residuals Plot"):
    residuals = y_true - y_pred
    plt.figure(figsize=(12, 6))
    plt.scatter(range(len(residuals)), residuals, color='orange', alpha=0.6)
    plt.axhline(0, color='black', linestyle='--', linewidth=2)  # Horizontal line at 0 for reference
    plt.title(title)
    plt.xlabel("Sample Index")
    plt.ylabel("Residuals (Actual - Predicted)")
    plt.show()

### Evaluate metrics

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

def evaluate_metrics(y_train, y_pred_train, y_test, y_pred_test):
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_mape = mean_absolute_percentage_error(y_train, y_pred_train)
    test_mape = mean_absolute_percentage_error(y_test, y_pred_test)

    print("-------------------------------------------------------------------------------")
    print(f"Training RMSE: {train_rmse}, Training R^2: {train_r2}")
    print(f"Testing RMSE: {test_rmse}, Testing R^2: {test_r2}")
    print(f"Training MAE: {train_mae}, Testing MAE: {test_mae}")
    print(f"Training MAPE: {train_mape * 100:.2f}%, Testing MAPE: {test_mape * 100:.2f}%")
    print("-------------------------------------------------------------------------------")

In [ ]:
def evaluate_metrics_naive(y_test, y_pred):
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  r2 = r2_score(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  mape = mean_absolute_percentage_error(y_test, y_pred)*100

  print(f"RMSE: {rmse}")
  print(f"R-squared: {r2}")
  print(f"MAE: {mae}")
  print(f"MAPE: {mape}")

## **6.1 Baseline Models**

###  **Naïve**

In [ ]:
# Naive model
class NaiveForecast:
  def __init__(self):
    self.last_value = None  # Store the last observed value for forecasting

  def fit(self, series):
    self.last_value = series.iloc[-1]  # The last value of the series

  def predict(self, n_periods=1):
    if self.last_value is None:
        raise ValueError("Model is not fitted yet. Call fit() before predict().")
    return [self.last_value] * n_periods


In [ ]:
import time

def pred_naive(y_train, y_test):
  start_time = time.time()
  naive_model = NaiveForecast()
  naive_model.fit(y_train)

  naive_predictions = naive_model.predict(n_periods=len(y_test))
  name = "Naive"
  end_time = time.time()
  execution_time = end_time - start_time
  return naive_predictions, name, execution_time

In [ ]:
y_train, y_test = train_test_split(y_cleaned, test_size=0.2, random_state=42)

# Make predictions using the Naive model
naive_predictions, model_name, execution_time = pred_naive(y_train, y_test)

evaluate_metrics_naive(y_test, naive_predictions)
print(f'Execution time of the naive model is: {execution_time}')

In [ ]:
plot_forecast_vs_actual(y_test, naive_predictions, title="Forecasted vs Actual Values", num_samples=200)

plot_residuals(y_test, naive_predictions)

### **Moving Average**

In [ ]:
class MovingAverageForecast:
  def __init__(self, window_size):
    self.window_size = window_size
    self.history = None

  def fit(self, series):
    self.history = series.tolist() #.iloc[-self.window_size:]

  def predict(self, n_periods=1):
    if self.history is None:
        raise ValueError("Model is not fitted yet. Call fit() before predict().")
    predictions = []
    temp_history = self.history.copy()  # Create a copy of the history to simulate rolling predictions
    for _ in range(n_periods):
        if len(temp_history) < self.window_size:
            raise ValueError("Not enough history to calculate moving average.")
        # Compute the moving average of the last `window_size` values
        prediction = np.mean(temp_history[-self.window_size:])
        predictions.append(prediction)
        temp_history.append(prediction)  # Update the history for rolling predictions
    return np.array(predictions)

In [ ]:
from sklearn.metrics import mean_squared_error

def pred_moving_average_grid_search(y_train, y_test, window_sizes=[1, 3, 5, 7, 10, 14, 21, 30]):
    best_window_size = None
    best_score = float('inf')

    for window_size in window_sizes:
        moving_avg_model = MovingAverageForecast(window_size=window_size)
        moving_avg_model.fit(y_train)
        predictions = moving_avg_model.predict(n_periods=len(y_test))

        score = mean_squared_error(y_test, predictions)
        if score < best_score:
            best_score = score
            best_window_size = window_size

    start_time = time.time()
    final_model = MovingAverageForecast(window_size=best_window_size)
    final_model.fit(y_train)
    final_predictions = final_model.predict(n_periods=len(y_test))
    end_time = time.time()

    name = f"Moving Average (Best Window Size: {best_window_size})"
    execution_time = end_time - start_time
    return final_predictions, name, execution_time


In [ ]:
y_train, y_test = train_test_split(y_cleaned, test_size=0.2, random_state=42)

# Make predictions using the moving avg model
ma_predictions, model_name, execution_time = pred_moving_average_grid_search(y_train, y_test)

evaluate_metrics_naive(y_test, ma_predictions)
print(f'Execution time of the moving average model is: {execution_time}')

In [ ]:
plot_forecast_vs_actual(y_test, ma_predictions, title="Forecasted vs Actual Values", num_samples=200)

plot_residuals(y_test, ma_predictions)

### **Seasonal Naïve**

In [ ]:
import numpy as np

class SeasonalNaiveForecast:
  def __init__(self, season_length):
      self.return_value = None
      self.season_length = season_length

  def fit(self, series):
      self.return_value = series.iloc[-self.season_length:].values

  def predict(self, n_periods=1):
      if self.return_value is None:
          raise ValueError("Model is not fitted yet. Call fit() before predict().")
      predictions = []
      for i in range(n_periods):
          predictions.append(self.return_value[i % self.season_length])
      return np.array(predictions)

In [ ]:
def pred_naive_seasonal_grid_search(y_train, y_test, season_lengths = [1, 3, 5, 7, 10, 14, 21, 30]):
  best_season_length = None
  best_score = float('inf')

  for season_length in season_lengths:
    seasonal_naive_model = SeasonalNaiveForecast(season_length=season_length)
    seasonal_naive_model.fit(y_train)
    predictions = seasonal_naive_model.predict(n_periods=len(y_test))

    score = mean_squared_error(y_test, predictions)
    if score < best_score:
        best_score = score
        best_season_length = season_length

  start_time = time.time()
  final_model = SeasonalNaiveForecast(season_length=best_season_length)
  final_model.fit(y_train)
  final_predictions = final_model.predict(n_periods=len(y_test))
  end_time = time.time()

  name = f"Seasonal Naive (Best Season Length: {best_season_length})"
  execution_time = end_time - start_time
  return final_predictions, name, execution_time


In [ ]:
y_train, y_test = train_test_split(y_cleaned, test_size=0.2, random_state=42)

# Make predictions using the seasonal naive model
seasonal_naive_preds, model_name, execution_time = pred_naive_seasonal_grid_search(y_train, y_test)

evaluate_metrics_naive(y_test, seasonal_naive_preds)
print(f'Execution time of the naive model is: {execution_time}')

In [ ]:
plot_forecast_vs_actual(y_test, seasonal_naive_preds, title="Forecasted vs Actual Values", num_samples=200)

plot_residuals(y_test, seasonal_naive_preds)

## **6.3 Machine Learning Models**

###  **XGBOOST**

In [ ]:
# Feature scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cleaned, test_size=0.2, random_state=42)

start_time = time.time()

# 2. Train XGBoost model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Fit model
xgb_model.fit(X_train, y_train)

# 3. Model Evaluation
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

end_time = time.time()

# Metrics
evaluate_metrics(y_train, y_pred_train, y_test, y_pred_test)

# Feature Importance
importance = xgb_model.feature_importances_

min_length = min(len(new_features), len(importance))
feature_importance = pd.DataFrame({'Feature': new_features[:min_length], 'Importance': importance[:min_length]})
print(feature_importance.sort_values(by='Importance', ascending=False))
execution_time = end_time - start_time
print(f"XGBoost execution time: {execution_time} seconds")

In [ ]:
#Real vs predicted
plot_forecast_vs_actual(y_test, y_pred_test, title="Forecasted vs Actual Values (First 200 Samples)")

In [ ]:
# Residuals
plot_residuals(y_test, y_pred_test, title="Residuals of Forecasted vs Actual Values (Test Set)")

### **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load and preprocess the dataset
# Handle missing values (if any)
X.fillna(X.mean(), inplace=True)

# Handle outliers using the function and print the count of outliers replaced

# # Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# # Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cleaned, test_size=0.2, random_state=42)

start_time = time.time()

# Initialize Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=500,  # Number of trees
    max_depth=None,    # Expand until all leaves are pure
    random_state=42,
    n_jobs=-1          # Utilize all processors
)

# Fit the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

end_time = time.time()

# Evaluate the model
evaluate_metrics(y_train, y_pred_train, y_test, y_pred_test)

min_length = min(len(new_features), len(rf_model.feature_importances_))
feature_importance = pd.DataFrame({'Feature': new_features[:min_length], 'Importance': rf_model.feature_importances_[:min_length]})
print(feature_importance.sort_values(by='Importance', ascending=False))
execution_time = end_time - start_time
print(f"Random Forest execution time: {execution_time} seconds")

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual(y_test, y_pred_test, title="Forecasted vs Actual Values (Test Set)")

In [ ]:
# Plot residuals for the test set
plot_residuals(y_test, y_pred_test, title="Residuals of Forecasted vs Actual Values (Test Set)")

### **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,mean_absolute_percentage_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


# Define Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)

# Define parameter grid for Grid Search
param_grid = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': [None, 'sqrt', 'log2']
}

start_time = time.time()

# Perform Grid Search
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Best model and parameters
best_dt_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Fit the best model
best_dt_model.fit(X_train, y_train)

# Make predictions
y_pred_train = best_dt_model.predict(X_train)
y_pred_test = best_dt_model.predict(X_test)

end_time = time.time()

# Evaluate the model
evaluate_metrics(y_train, y_pred_train, y_test, y_pred_test)
execution_time = end_time - start_time
print(f"Decision Tree execution time: {execution_time} seconds")

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual(y_test, y_pred_test, title="Forecasted vs Actual Values (Test Set)")

# Plot residuals for the test set
plot_residuals(y_test, y_pred_test, title="Residuals of Forecasted vs Actual Values (Test Set)")

## **6.4 Deep Learning Models**

### **LSTM (Long Short-Term Memory)**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class for LSTM
class EnergyDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, index):
        X_seq = self.X[index:index + self.seq_length]
        y_seq = self.y[index + self.seq_length - 1]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Train-test split
seq_length = 24
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False)

train_dataset = EnergyDataset(X_train, y_train, seq_length)
test_dataset = EnergyDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model parameters
input_size = X.shape[1]
hidden_size = 64
num_layers = 2
output_size = 1

# Initialize model
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

# Training
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        y_pred.extend(outputs.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

# Inverse scaling
y_pred = scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
y_true = scaler.inverse_transform(np.array(y_true).reshape(-1, 1))

end_time = time.time()

# Metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, mape, r2

mse, rmse, mae, mape, r2 = calculate_metrics(y_true, y_pred)
print(f'RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}, R^2: {r2:.4f}')
execution_time = end_time - start_time
print(f"LSTM execution time: {execution_time} seconds")

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual_dl(y_true, y_pred, title="Forecasted vs Actual Values (Test Set)")

# Plot residuals for the test set
plot_residuals(y_true, y_pred, title="Residuals of Forecasted vs Actual Values (Test Set)")

### **GRU (Gated Recurrent Unit)**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class for GRU
class EnergyDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, index):
        X_seq = self.X[index:index + self.seq_length]
        y_seq = self.y[index + self.seq_length - 1]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Train-test split
seq_length = 24
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False)

train_dataset = EnergyDataset(X_train, y_train, seq_length)
test_dataset = EnergyDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model parameters
input_size = X.shape[1]
hidden_size = 64
num_layers = 2
output_size = 1

# Initialize model
model = GRUModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

# Training
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        y_pred.extend(outputs.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

# Inverse scaling
y_pred = scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
y_true = scaler.inverse_transform(np.array(y_true).reshape(-1, 1))

end_time = time.time()

# Metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, mape, r2

mse, rmse, mae, mape, r2 = calculate_metrics(y_true, y_pred)
print(f'RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}, R^2: {r2:.4f}')
execution_time = end_time - start_time
print(f"GRU execution time: {execution_time} seconds")

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual_dl(y_true, y_pred, title="Forecasted vs Actual Values (Test Set)")

# Plot residuals for the test set
plot_residuals(y_true, y_pred, title="Residuals of Forecasted vs Actual Values (Test Set)")

### **CNNs for Time Series (TCN)**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class for TCN model
class EnergyDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, index):
        X_seq = self.X[index:index + self.seq_length]
        y_seq = self.y[index + self.seq_length - 1]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# TCN Model with dilated convolutions
class TCNModel(nn.Module):
    def __init__(self, input_size, seq_length, num_filters, kernel_size, output_size, num_layers=5):
        super(TCNModel, self).__init__()
        self.num_layers = num_layers
        self.conv_layers = nn.ModuleList()

        # Add TCN layers with dilated convolutions
        for i in range(num_layers):
            dilation = 2 ** i
            self.conv_layers.append(
                nn.Conv1d(in_channels=input_size if i == 0 else num_filters,
                          out_channels=num_filters,
                          kernel_size=kernel_size,
                          padding=dilation,
                          dilation=dilation)
            )
            # Batch normalization and ReLU activation
            self.conv_layers.append(nn.BatchNorm1d(num_filters))
            self.conv_layers.append(nn.ReLU())

        self.fc = nn.Linear(num_filters, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)

        for conv in self.conv_layers:
            x = conv(x)

        x = x.mean(dim=2)
        x = self.fc(x)
        return x

# Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Train-test split
seq_length = 24
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False)

train_dataset = EnergyDataset(X_train, y_train, seq_length)
test_dataset = EnergyDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model parameters
input_size = X.shape[1]
num_filters = 64
kernel_size = 3
output_size = 1

# Initialize model
model = TCNModel(input_size, seq_length, num_filters, kernel_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        y_pred.extend(outputs.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

# Inverse scaling
y_pred = scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
y_true = scaler.inverse_transform(np.array(y_true).reshape(-1, 1))

# Metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, mape, r2

mse, rmse, mae, mape, r2 = calculate_metrics(y_true, y_pred)
print(f'RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}, R^2: {r2:.4f}')

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual_dl(y_true, y_pred, title="Forecasted vs Actual Values (Test Set)")

# Plot residuals for the test set
plot_residuals(y_true, y_pred, title="Residuals of Forecasted vs Actual Values (Test Set)")

## **6.5 Transformer Models**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class for Transformer
class EnergyDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, index):
        X_seq = self.X[index:index + self.seq_length]
        y_seq = self.y[index + self.seq_length - 1]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, seq_length, output_size, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_length, d_model))
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout),
            num_encoder_layers
        )
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding
        x = self.encoder(x.permute(1, 0, 2))  # Transformer expects (seq_length, batch_size, d_model)
        x = self.fc(x[-1])  # Use the last time step
        return x

# Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Train-test split
seq_length = 24
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False)

train_dataset = EnergyDataset(X_train, y_train, seq_length)
test_dataset = EnergyDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model parameters
input_size = X.shape[1]
d_model = 64
nhead = 4
num_encoder_layers = 3
dim_feedforward = 128
output_size = 1
dropout = 0.1

# Initialize model
model = TransformerModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, seq_length, output_size, dropout).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

# Training
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        y_pred.extend(outputs.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

# Inverse scaling
y_pred = scaler.inverse_transform(np.array(y_pred).reshape(-1, 1))
y_true = scaler.inverse_transform(np.array(y_true).reshape(-1, 1))

end_time = time.time()

# Metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, mape, r2

mse, rmse, mae, mape, r2 = calculate_metrics(y_true, y_pred)
print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}, R^2: {r2:.4f}')
execution_time = end_time - start_time
print(f"Transformer execution time: {execution_time} seconds")

In [ ]:
# Plot forecasted vs actual values for the test set
plot_forecast_vs_actual_dl(y_true, y_pred, title="Forecasted vs Actual Values (Test Set)")

# Plot residuals for the test set
plot_residuals(y_true, y_pred, title="Residuals of Forecasted vs Actual Values (Test Set)")